In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
import tensorflow as tf

# Cargar el modelo completo (estructura + pesos + configuración)
model = tf.keras.models.load_model('/content/drive/MyDrive/BARRAS_DOCTORADO/modelo_entrenadoj.h5')

# Nombres de las emociones y valores normalizados máximos y mínimos
labels = ["Angry", "Contempt", "Disgusted", "Happy", "Neutral", "Sad", "Scared", "Surprised"]
max_values = [5.138851, 4.523557, 7.737973, 5.354393, 1.869215, 8.036036, 9.170244, 2.616412]
min_values = [-0.548240, -1.044128, -0.379009, -0.520728, -1.589441, -0.545767, -0.416786, -0.853029]

# Crear barras deslizantes para las emociones en una disposición más vertical
sliders = [
    widgets.FloatSlider(
        value=(min_val + max_val) / 2,
        min=min_val,
        max=max_val,
        step=0.00001,  # Ajuste para permitir 5 decimales
        description=label,
        readout_format='.5f',  # Mostrar cinco decimales
        style={'description_width': '120px'},  # Ajustar el tamaño de la descripción
        layout=widgets.Layout(width='90%', height='40px')
    )  # Paréntesis de cierre añadido aquí
    for label, min_val, max_val in zip(labels, min_values, max_values)  # Usar max_values en lugar de min_values
]

# Insertar CSS personalizado para mejorar la apariencia de las barras deslizantes y la interfaz
css = """
<style>
    /* General Styles */
    .widget-label {
        font-weight: bold;
        font-size: 14px;
        color: #34495e; /* Dark gray */
    }

    .slider-container .slider, .slider-container .slider-track, .slider-container .slider-thumb {
        height: 12px;
        background-color: #1abc9c; /* Soft teal */
    }

    .widget-readout {
        font-size: 12px;
        color: #2c3e50; /* Dark blue */
    }

    /* FloatSlider Styling */
    .p-Widget.p-FloatSlider .slider-thumb {
        background-color: #2980b9; /* Dark blue */
        border-radius: 50%;
        border: 2px solid #2980b9;
    }

    .p-Widget.p-FloatSlider .slider-track {
        background-color: #bdc3c7; /* Light gray for track */
        height: 10px;
        border-radius: 10px;
    }

    .p-Widget.p-FloatSlider .slider {
        background-color: #27ae60; /* Progress color */
        border-radius: 10px;
        box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
    }

    /* Estilo de botón */
    .p-Widget.w-button {
        background-color: #3498db; /* Light blue */
        color: white;
        border-radius: 12px;
        padding: 8px;
        font-weight: bold;
        box-shadow: 0px 4px 10px rgba(0,0,0,0.2); /* Add shadow */
    }

    .p-Widget.w-button:hover {
        background-color: #2980b9; /* Darker blue */
    }

    /* Barra de progreso */
    .p-Widget.p-IntProgress {
        background-color: #bdc3c7; /* Medium gray */
        border-radius: 8px;
        height: 12px;
    }

    .p-Widget.p-IntProgress .progress-bar {
        background-color: #27ae60; /* Green */
        height: 12px;
    }

    /* Spinner de carga */
    .loading-spinner {
        display: none;
        justify-content: center;
        align-items: center;
    }

    /* Estilo del mensaje de resultado */
    .result-message {
        font-size: 16px;  /* Tamaño más pequeño */
        font-weight: bold;
        padding: 5px 10px;
        border-radius: 8px;
        text-align: center;
        margin-top: 10px;
        display: inline-block;
        width: 150px;  /* Tamaño más compacto */
    }

    .result-pass {
        background-color: #2ecc71; /* Soft green */
        color: white;
        border: 2px solid #27ae60;
    }

    .result-fail {
        background-color: #e74c3c; /* Soft red */
        color: white;
        border: 2px solid #c0392b;
    }

    /* Contenedor principal */
    .main-container {
        padding: 20px;
        border-radius: 15px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        background-color: #ecf0f1; /* Light gray */
    }

    .header {
        font-size: 24px;
        font-weight: bold;
        color: #2c3e50;
        text-align: center;
        margin-bottom: 15px;
    }

    .section {
        margin-bottom: 15px;
        padding: 10px;
    }
</style>
"""
display(HTML(css))

# Botón para realizar predicciones
predict_button = widgets.Button(
    description="Predecir",
    style={'button_color': '#3498db'},
    layout=widgets.Layout(width='50%', height='40px')
)

# Barra de progreso para mostrar la probabilidad
probability_bar = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='Probabilidad:',
    bar_style='',  # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout=widgets.Layout(width='80%', height='12px')  # Barra de progreso más fina
)

# Área de salida para mostrar los resultados de la predicción
output = widgets.Output(layout={'border': '2px solid #34495e', 'padding': '10px', 'width': '50%', 'margin': 'auto'})

# Spinner de carga
loading_spinner = widgets.HTML(
    value="<img src='https://i.gifer.com/ZZ5H.gif' width='30px' height='30px'/>",
    layout=widgets.Layout(display='none')
)

# Contenedor para el mensaje de aprobación/desaprobación
result_text = widgets.HTML(value="", layout=widgets.Layout(display='none'))

# Función para manejar el evento de clic en el botón de predicción
def on_predict_button_clicked(b):
    loading_spinner.layout.display = 'flex'
    result_text.layout.display = 'none'
    with output:
        output.clear_output()
        inputs = [slider.value for slider in sliders]  # Solo las emociones
        prediction = model.predict(np.array([inputs]))[0][0]  # Realiza la predicción con el modelo cargado
        binary_classification = int(prediction > 0.5)

        color = 'lightgreen' if binary_classification == 1 else '#F1948A'
        output.layout.border = f'2px solid {color}'
        output.layout.background_color = color

        # Actualizar la barra de progreso y el texto según la clasificación
        probability_bar.value = prediction
        if binary_classification == 1:
            result_text.value = "<div class='result-message result-pass'>¡Aprueba!</div>"
        else:
            result_text.value = "<div class='result-message result-fail'>¡No aprueba!</div>"

        result_text.layout.display = 'block'

        print(f"Probabilidad predicha: {prediction:.5f}, Clasificación binaria: {binary_classification}")
    loading_spinner.layout.display = 'none'

predict_button.on_click(on_predict_button_clicked)

# Mostrar los controles deslizantes, el botón y el área de salida
grid = widgets.VBox(
    children=[
        widgets.HTML("<div class='header'>Predicción del Desempeño</div>"),  # Título
        widgets.HTML("<div class='section'>Emociones:</div>"),
        widgets.VBox(sliders, layout=widgets.Layout(width='100%')),
        predict_button,
        widgets.HTML("<div class='section'>Probabilidad de Aprobación:</div>"),
        probability_bar,
        loading_spinner,
        output,
        result_text
    ],
    layout=widgets.Layout(padding="20px", border="2px solid #3498db", border_radius="15px", width='80%', margin='0 auto'))

# Mostrar la interfaz completa
display(grid)